# Chains in LangChain

Chains in LangChain helps to chain all other components & other chains together in a sequence.

This course is using *little old version of LangChain*, although there is a new syntax called "LCEL" (langchain expression language) for defining chains effectively!!

Let's dive into that.

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

# Environment variables
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo" # This..
else:
    llm_model = "gpt-3.5-turbo-0301"

# For me it was "gpt-3.5-turbo" model

In [4]:
#!pip install pandas

In [5]:
# Importing data and creating a DataFrame
# we will use this data for creating longer chains later!

import pandas as pd
df = pd.read_csv('Data.csv') # read csv file.

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [17]:
# Downloading the file for later use...
# !pip3 install IPython -U

from IPython.display import FileLink

file_path = 'Data.csv'
FileLink(file_path)

/home/jovyan/work/L3-Chains/Data.csv

## LLMChain

This is a legacy simple chain, that chains together an `LLM` and `Prompt Template` together, and provides an output on run! But it's deprecated now!

Instead we can use `|` symbol (LCEL), that is suggested! [See Here](https://python.langchain.com/docs/tutorials/llm_chain/#chaining-together-components-with-lcel).

In [18]:
# Importing 
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain # instead we can use `|` symbol

In [19]:
# Instantiate the model
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [20]:
# Prompt template with a "product" as input.
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [21]:
# Creating a basic chain!
chain = LLMChain(llm=llm, prompt=prompt)

In [22]:
# Running the chain with input!
product = "Queen Size Sheet Set"
chain.run(product)

'"Regal Comfort Linens"'

In [23]:
# Let's try more
chain.run("Sher ka shikaar") #😆

'"King\'s Hunt"'

## Sequential Chains

Sequential chains are Chains that run other lots of chains in a sequence, and get us the final output! They are in two forms:

1. **SimpleSequentialChain** : For combining chains with single input/output
2. **SequencialChain** : For chains having multiple inputs/outputs

### SimpleSequentialChain

Let's combine two simple chains with single input/output as a sequence using `SimpleSequentialChain`.

In [25]:
from langchain.chains import SimpleSequentialChain

In [26]:
# LLM
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt) 

In [27]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [28]:
# Creating Simple sequential chain by combining them!
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True # See workflow
                                            )

In [29]:
# Running the chain!
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Regal Comfort Linens"
Regal Comfort Linens offers luxurious and high-quality bedding, sheets, and blankets to ensure a cozy and comfortable night's sleep.

> Finished chain.


"Regal Comfort Linens offers luxurious and high-quality bedding, sheets, and blankets to ensure a cozy and comfortable night's sleep."

## SequentialChain

Now, if we have to combine chains with multiple different inputs and outputs?!
`SequentialChain` is required here!

In [31]:
# Let's import that!
from langchain.chains import SequentialChain

In [32]:
# First translate review to english
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review" # Key
                    )


In [33]:
# Summarize the translated review in a sentence
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [34]:
# prompt template 3: Find language of the original review.
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [35]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [43]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [44]:
review = df.Review[3]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'This is the best throw pillow fillers on Amazon. I’ve tried several others, and they’re all cheap and flat no matter how much fluffing you do. Once you toss these in the dryer after you remove them from the vacuum sealed shipping material, they fluff up great',
 'English_Review': 'Esta es la mejor relleno para cojines en Amazon. He probado varios otros, y todos son baratos y planos sin importar cuánto los revuelvas. Una vez que los metes en la secadora después de sacarlos del empaque sellado al vacío de envío, se inflan muy bien.',
 'summary': 'Este relleno para cojines de Amazon es el mejor, ya que se infla completamente después de meterlo en la secadora.',
 'followup_message': 'Thank you for sharing your experience with the cushion filler from Amazon. I appreciate knowing that it fully inflates after being put in the dryer. I will definitely consider purchasing it for myself.'}

## Router Chain

Router chains are used to build more complex workflows, where we don't explicitly know what subchain to call next!

Router chain routes the application logic based on the given context automatically and calls the most appropriate chain itself using a Language Model under the hood!

In [45]:
# Let's create multiple Prompt templates to choose from (to Route in)

physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [46]:
# This prompt info (name, description, actual template) is given
# to the Router template, to know where & why to route!
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [47]:
# Import statements
from langchain.chains.router import MultiPromptChain
 # Route using LLM Reasoning & parse it's Output
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [48]:
# LLM to be used!
llm = ChatOpenAI(temperature=0, model=llm_model)

In [49]:
# Storing individual chains in `destination_chains`
# Also formatting `destinations_str` to provide in Router prompt

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    
    # Create and store corresponding chains
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  

# Create destination description strings
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [50]:
# A default prompt, in case Router cannot decide it!
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [52]:
# Template for the Router to decid
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [54]:
# Formating router template with destination_str
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

# Router prompt for the LLM!
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

# Router chain
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [55]:
# Final `MultiPromptChain`, given all router_chain, destination_chains & default_chain
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [57]:
# Running & see what sub-chain it chooses
chain.run("What is event horizen in a black hole??")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is event horizon in a black hole?'}
> Finished chain.


"The event horizon of a black hole is the point of no return beyond which nothing, not even light, can escape the gravitational pull of the black hole. It is essentially the boundary surrounding a black hole where the escape velocity is equal to the speed of light. Once an object crosses the event horizon, it is inevitably pulled into the black hole's singularity at the center."

In [58]:
chain.run("what is least squares?")



> Entering new MultiPromptChain chain...
math: {'input': 'what is least squares?'}
> Finished chain.


'Least squares is a method used in statistics and mathematics to find the best-fitting line or curve for a set of data points. It works by minimizing the sum of the squares of the differences between the observed values and the values predicted by the model. This method is commonly used in regression analysis to estimate the relationship between variables and make predictions based on that relationship.'

In [59]:
# Maybe No destination will answer it! hence NONE!
chain.run("What is the meaning of life?! why life is absurd!?")



> Entering new MultiPromptChain chain...
None: {'input': 'What is the meaning of life?! why life is absurd!?'}
> Finished chain.


'The meaning of life is a deeply philosophical question that has been debated by thinkers and scholars for centuries. Some believe that the meaning of life is to seek happiness, fulfillment, or spiritual enlightenment, while others argue that life has no inherent meaning and it is up to each individual to create their own purpose.\n\nThe idea that life is absurd stems from existentialist philosophy, which posits that the universe is indifferent to human existence and that life has no inherent meaning or purpose. This can lead to feelings of nihilism and despair, as individuals grapple with the idea that their existence is ultimately meaningless.\n\nHowever, some existentialist thinkers argue that the absurdity of life can be liberating, as it allows individuals to create their own meaning and purpose in a world that is inherently chaotic and unpredictable. By embracing the absurdity of life, individuals can find freedom and autonomy in shaping their own destiny.\n\nUltimately, the mean

Reminder: Download your notebook to you local computer to save your work.